In [1]:
import pandas as pd
import numpy as np

In [5]:
filename='report_2016_09_01-2016_12_01'

In [2]:
df_test=pd.read_csv('report_2016_02_01-2016_12_01.csv.gz',nrows=100000)

In [24]:
reader = pd.read_csv('report_2016_02_01-2016_12_01.csv', chunksize=1000000)

In [28]:
cat_types

array(['Ad group ID', 'Ad group', 'Brand', 'Campaign ID', 'Campaign',
       'Category (1st level)', 'Category (2nd level)',
       'Category (3rd level)', 'Category (4th level)',
       'Category (5th level)', 'Country/Territory', 'Custom label 0',
       'Custom label 1', 'Custom label 2', 'Custom label 3',
       'Custom label 4', 'Device', 'Item Id', 'Product type (1st level)',
       'Product type (2nd level)', 'Product type (3rd level)',
       'Product type (4th level)', 'Product type (5th level)'], dtype=object)

In [29]:
dfs=[]
for chunk in reader:
    df=cast_categ(chunk,cat_types)
    dfs.append(cast_categ)

/home/sean/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2821: DtypeWarning: Columns (13,26) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/home/sean/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2821: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [30]:
df_all=pd.concat(dfs)

TypeError: cannot concatenate a non-NDFrame object

In [12]:
!gunzip -c report_2016_09_01-2016_12_01.csv.gz|wc

OSError: [Errno 12] Cannot allocate memory

In [47]:
df1=pd.read_csv('report_2016_09_01-2016_12_01.csv', nrows=2000000)

In [20]:
def cast_categ(df_inp, cat_types):
    for c in cat_types:
        df_inp[c]=df_inp[c].astype('category')

In [48]:
cast_categ(df1,cat_types)

In [49]:
df1.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
Ad group ID,2e+06,2576,3.35849e+10,54572,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Ad group,2000000,1539,marc o'polo,82865,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Brand,2000000,1114,marc o'polo,84825,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Campaign ID,2e+06,6,6.44233e+08,1.25694e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Campaign,2000000,6,camato Designer,1256943,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Category (1st level),2000000,11,Bekleidung & Accessoires,1702086,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Category (2nd level),2000000,34,Bekleidung,1351333,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Category (3rd level),2000000,69,Überbekleidung,439678,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Category (4th level),2000000,93,--,1137782,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Category (5th level),2000000,55,--,1847175,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [51]:
cnt=df1.groupby([ 'Ad group', 'Ad group ID']).size()

In [52]:
cnt.shape

(2576,)

In [45]:
cnt.reset_index().describe()

ValueError: Buffer dtype mismatch, expected 'Python object' but got 'long'

In [53]:
cnt1=cnt.groupby(level='Ad group').size()

In [54]:
cnt1[cnt1>1]

Ad group
0039 italy                            2
120%lino                              2
4lou                                  2
7 for all mankind                     2
81hours                               2
a beautiful story                     2
a-k-r-i-s-                            2
abro                                  2
acca kappa                            2
acqua di parma                        2
adenauer & co                         2
adidas                                2
adidas by raf simons                  2
adidas by stella mccartney            2
adidas originals                      2
adidas stellasport                    2
aeronautica militare                  2
ag jeans                              2
agl attilio giusti leombruni          2
agl-attilio giusti leombruni          2
aglini                                2
agnona                                2
aigner                                2
airfield                              2
airforce                       

In [38]:
cnt.head()

Ad group         Ad group ID
Marken_Barts     17602163553      125
Marken_Boss      17601240513    12412
Marken_Cambio    17602161273      424
Marken_Desigual  17922688713      174
Marken_Doell     17602162953       57
dtype: int64

In [36]:
cnt.reset_index()['Ad group'].isnull().sum()

0

In [32]:
df_test[['Ad group ID', 'Ad group', 'Brand']].sample(50)

,Ad group ID,Ad group,Brand
99468,17601222753,Marken_Marc-Cain,marccain
9072,17601240513,Marken_Boss,boss
71515,17601229593,Marken_Marc-O'Polo,marc o'polo
24755,17601222753,Marken_Marc-Cain,marccain
75791,17601222753,Marken_Marc-Cain,marccain
32600,17602244913,Marken_Rest,strenesse
22092,17601240513,Marken_Boss,boss
24520,17602244913,Marken_Rest,raffaello rossi
29028,17601230193,Marken_Drykorn,drykorn
94651,17601229593,Marken_Marc-O'Polo,marc o'polo pure


In [26]:
dt=pd.read_csv(filename+'_datadict.csv',usecols=range(2))
cat_types=dt.column_name[dt.data_type=='category'].values

In [27]:
cat_dict={c:'category' for c in cat_types}

In [9]:
for c in dt.column_name[dt.data_type=='category']:
    df_test[c]=df_test[c].astype('category')

In [10]:
df_test.describe().T

,count,mean,std,min,25%,50%,75%,max
Clicks,100000.0,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0
Conversions,100000.0,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0
Total conv. value,100000.0,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0
Cost,100000.0,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0
Country/Territory,100000.0,2276.00000,0.000000,2276.0,2276.0,2276.0,2276.0,2276.0
Cross-device conv.,100000.0,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0
Impressions,100000.0,0.00003,0.005477,0.0,0.0,0.0,0.0,1.0


In [7]:
dt

,column_name,data_type
0,Ad group ID,category
1,Ad group,category
2,Brand,category
3,Campaign ID,category
4,Campaign,category
5,Category (1st level),category
6,Category (2nd level),category
7,Category (3rd level),category
8,Category (4th level),category
9,Category (5th level),category
